In [ ]:
#Setup Fenicsx in real mode to solve PDEs.
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
# Import nescessarry python modules.
import numpy as np
import pandas as pd
import ufl
import functools
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx import fem, mesh, plot, io
from dolfinx.fem import petsc as femPetsc
from scipy.io import savemat
from scipy.io import loadmat
from datetime import datetime

# Mount google drive storage to notebook.
from google.colab import drive
drive.mount('/content/gdrive/')

mainProjectDirGDrive = "/content/gdrive/MyDrive/Github/HJDQN/HJQ/"

pde_solutions_folder = mainProjectDirGDrive + "pde_outputs/pde_solutions/"
pde_fem_matrices_folder = mainProjectDirGDrive + "pde_outputs/fem_matrices/"
pde_ricatti_solution_matrices = mainProjectDirGDrive + "pde_outputs/ricatti_solution_matrices/"

In [ ]:
# Two dimensional PDE domain, boundary conditions and initial condition.

# Define initial and end time value.
t = 0
T = 2.0

# Define space interval.

#2d
Omega = [np.array([0,0]), np.array([1,1])]

# Time step configuraton.
num_steps = 200
dt = T / num_steps

# Define mesh.

#One dimensional mesh.
nx = 32
ny = 32
domain = mesh.create_unit_square(MPI.COMM_WORLD, nx, ny)

# Define function spaces.
V = fem.FunctionSpace(domain, ("P", 1))

# Define Dirichlet Boundary Condition.
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

# Create initial condition for the state function and interpolate it in V.

class Initial_condition():

  def __init__(self):
      pass

  def __call__(self,x):
      values = np.zeros(x.shape,dtype=PETSc.ScalarType)
      values = 3*np.sin(np.pi*x[0])*np.sin(np.pi*x[1])
      return values

initial_condition = Initial_condition()

y_0c = fem.Function(V)
y_0c.name = "y_0"
y_0c.interpolate(initial_condition)

y_0u = fem.Function(V)
y_0u.name = "y_0"
y_0u.interpolate(initial_condition)

In [ ]:
# PDE coefficients.

# Trial and test function.
yc, phic = ufl.TrialFunction(V), ufl.TestFunction(V)
yu, phiu = ufl.TrialFunction(V), ufl.TestFunction(V)

# PDE coefficients
nu = 0.1

class PDE_a():

  def __init__(self):
      pass

  def __call__(self,x):
      values = np.zeros(x.shape,dtype=PETSc.ScalarType)
      values = np.sin(x[0])
      return values

class PDE_b_1():

  def __init__(self):
      pass

  def __call__(self,x):
      values = np.zeros(x.shape,dtype=PETSc.ScalarType)
      values = -1.0*(x[0]+x[1])
      return values

class PDE_b_2():

  def __init__(self):
      pass

  def __call__(self,x):
      values = np.zeros(x.shape,dtype=PETSc.ScalarType)
      values = x[0]*x[1]
      return values

a_V = fem.Function(V)
a_V.interpolate(PDE_a())

b_V1 = fem.Function(V)
b_V2 = fem.Function(V)
b_V1.interpolate(PDE_b_1())
b_V2.interpolate(PDE_b_2())
b_V = ufl.as_vector([b_V1, b_V2])

a_Name = "sin(x[0])"
b_Name = "[-(x[0]+x[1]),x[0]*x[1]]"

In [ ]:
# PDE acctuators.

# Indicator function for 2d PDE also called acctuator functions.
w1 = np.array([[0.1,0.3],[0.1,0.3]])
w2 = np.array([[0.4,0.6],[0.1,0.3]])
w3 = np.array([[0.7,0.9],[0.1,0.3]])
w4 = np.array([[0.1,0.3],[0.4,0.6]])
w5 = np.array([[0.4,0.6],[0.4,0.6]])
w6 = np.array([[0.7,0.9],[0.4,0.6]])
w7 = np.array([[0.1,0.3],[0.7,0.9]])
w8 = np.array([[0.4,0.6],[0.7,0.9]])
w9 = np.array([[0.7,0.9],[0.7,0.9]])
tol = 1e-14

acctuatorDomains = [w1, w2, w3, w4, w5, w6, w7, w8, w9]

I_w = [fem.Function(V) for i in range(0,len(acctuatorDomains))]

for i, acct in enumerate(acctuatorDomains):
  I_w[i].interpolate(lambda x: np.where(np.logical_and(np.logical_and(x[0] - acct[0,0] + tol >= 0, acct[0,1] + tol - x[0] >= 0),np.logical_and(x[1] - acct[1,0] + tol >= 0, acct[1,1] + tol - x[1] >= 0)), 1, 0))

In [ ]:
# Assembling of matrices for care solver.

# Mass matrix
mass_form = yc*phic*ufl.dx
M_dx = femPetsc.assemble_matrix(fem.form(mass_form), bcs=[bc])
M_dx.assemble()
M_dx.convert("dense")
M = M_dx.getDenseArray()

# Stiffness matrix
stiffness_form = -1.0*( nu*dt*ufl.dot(ufl.grad(yc),ufl.grad(phic))*ufl.dx + dt*a_V*yc*phic*ufl.dx + dt*ufl.dot(b_V*yc,ufl.grad(phic))*ufl.dx)
A_dx = femPetsc.assemble_matrix(fem.form(stiffness_form), bcs=[bc])
A_dx.assemble()
A_dx.convert("dense")
A = A_dx.getDenseArray()

# Stiffness matrix for reward function
stiffness_form_reward = ufl.dot(ufl.grad(yc),ufl.grad(phic))*ufl.dx
A_reward_dx = femPetsc.assemble_matrix(fem.form(stiffness_form_reward), bcs=[bc])
A_reward_dx.assemble()
A_reward_dx.convert("dense")
A_reward = A_reward_dx.getDenseArray()

# State dimension
n = A.shape[0]

# Remove boundary
deleteRows = np.where(np.sum(M,axis=0)==1)
deleteCols = np.where(np.sum(M,axis=1)==1)
A_wb = np.delete(A,deleteRows,axis=0)
A_wb = np.delete(A_wb,deleteCols,axis=1)
A_reward_wb = np.delete(A_reward,deleteRows,axis=0)
A_reward_wb = np.delete(A_reward_wb,deleteCols,axis=1)
M_wb = np.delete(M,deleteRows,axis=0)
M_wb = np.delete(M_wb,deleteCols,axis=1)

# Acctuator matrix
Ac = np.array([I_w[0].x.array])
for i in range(1,len(I_w)):
  Ac = np.r_[Ac, [I_w[i].x.array]]
Ac = Ac.T
Ac = np.delete(Ac,deleteRows,axis=0)
B = np.matmul(M_wb,Ac)

# State dimension without boundary
n_wb = A_wb.shape[0]

# Action dimesion
m = B.shape[1]

# Matrices and coefficients for the reward function r = (alpha/2)*y'*Q*y + (beta/2)*u*R*u
alpha = 1.
beta = 5000.
Q_wb = (alpha/2)*A_reward_wb
Q = (alpha/2)*A_reward
R = (beta/2)*np.eye(m)

# CARE solution for exact solution in the linear pde case --------------------------------------------------------

# Continuous discount factor.
gamma = .99999
conti_gamma = (1 - gamma) / dt

# Solve discounted generalized CARE
Ad = A_wb - (conti_gamma / 2.) * np.eye(n_wb)

Omega_array = np.array(Omega)
acctuator = np.array(acctuatorDomains[0])
for i in range(1,len(acctuatorDomains)):
  acctuator = np.concatenate((acctuator, acctuatorDomains[i]))

fem_matrices = "fem_matrices{}_linear_PDE_2D.mat".format(datetime.now().strftime("_%Y-%m-%dT%H%M%S"))
mdic = {"M": M_wb, "Ad": Ad, "B": B, "Q": Q_wb, "R": R, "Omega": Omega_array, "n": n, "m": m, "num_steps": num_steps, "dt": dt, "nu": nu, "a_Name": a_Name, "b_Name": b_Name, "acctuators": acctuator, "alpha": alpha, "beta": beta, "deleteRows": deleteRows, "deleteCols": deleteCols}
savemat(f"{pde_fem_matrices_folder}{fem_matrices}", mdic)

In [ ]:
# Lookup Ricatti Solution Matrix Files
solution_dictionary = pd.read_csv("{}/ricatti_solution_dictonary.csv".format(pde_ricatti_solution_matrices))
solution_dictionary

In [ ]:
K_filename = "K_2023-07-09T131757"
K_filename_mat = "{}.mat".format(K_filename)
timestamp = K_filename[1:]
mat = loadmat(f"{pde_ricatti_solution_matrices}{K_filename_mat}")
K_mat = mat['K']

# Time stamp
exec_time = timestamp # datetime.now().strftime("_%Y-%m-%dT%H%M%S")

# Create output file.
pde_solutionc = "HJDQN{}_linear_PDE_2D_state.xdmf".format(exec_time)
pde_solutionu = "HJDQN{}_uncontrolled_linear_PDE_2D_state.xdmf".format(exec_time)

xdmfc = io.XDMFFile(domain.comm, f"{pde_solutions_folder}{pde_solutionc}", "w", encoding=io.XDMFFile.Encoding.HDF5)
xdmfc.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview.
y_nc = fem.Function(V)
y_nc.name = "y_n"
y_nc.interpolate(initial_condition)
xdmfc.write_function(y_nc, t)

# Control vector components.
us = [fem.Constant(domain, 0.0) for i in range(0,len(I_w))]

# Variational Problem (LHS).
a_hc = yc*phic*ufl.dx + nu*dt*ufl.dot(ufl.grad(yc),ufl.grad(phic))*ufl.dx + dt*a_V*yc*phic*ufl.dx - dt*ufl.dot(b_V*yc,ufl.grad(phic))*ufl.dx
bilinear_formc = fem.form(a_hc)

# Variational Problem (RHS).
Lc = us[0]*I_w[0]
for i in range(1,len(us)):
  Lc = Lc + us[i]*I_w[i]
Lc = (y_0c + dt*Lc)*phic*ufl.dx
linear_formc = fem.form(Lc)

# Assemble FEM matrices.
Cc = femPetsc.assemble_matrix(bilinear_formc)
Cc.assemble()
dc = femPetsc.create_vector(linear_formc)

# Create solver.
solverc = PETSc.KSP().create(domain.comm)
solverc.setOperators(Cc)
solverc.setType(PETSc.KSP.Type.PREONLY)
solverc.getPC().setType(PETSc.PC.Type.LU)

# Whole time intervall.
for i in range(num_steps):
  t += dt

  # Control vector.
  y_0_arr = y_0c.x.array
  u = -1*np.matmul(K_mat,np.delete(y_0_arr,deleteRows,axis=0))
  uls = u.tolist()

  # Update coefficients of control vector.
  for i in range(0,len(us)):
    us[i].value = uls[i]

  # Update the right hand side reusing the initial vector
  with dc.localForm() as loc_b:
    loc_b.set(0)
  femPetsc.assemble_vector(dc, linear_formc)

  # Apply Dirichlet boundary condition to the vector
  femPetsc.apply_lifting(dc, [bilinear_formc], [[bc]])
  dc.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
  femPetsc.set_bc(dc, [bc])

  # Solve linear problem
  solverc.solve(dc, y_nc.vector)
  y_nc.x.scatter_forward()

  # Update solution at previous time step
  y_0c.x.array[:] = y_nc.x.array

  # Write solution to file
  xdmfc.write_function(y_nc, t)
xdmfc.close()

t = 0.0
xdmfu = io.XDMFFile(domain.comm, f"{pde_solutions_folder}{pde_solutionu}", "w", encoding=io.XDMFFile.Encoding.HDF5)
xdmfu.write_mesh(domain)

# Define solution variable, and interpolate initial solution for visualization in Paraview.
y_nu = fem.Function(V)
y_nu.name = "y_n"
y_nu.interpolate(initial_condition)
xdmfu.write_function(y_nu, t)

# Variational Problem (LHS).
a_hu = yu*phiu*ufl.dx + nu*dt*ufl.dot(ufl.grad(yu),ufl.grad(phiu))*ufl.dx + dt*a_V*yu*phiu*ufl.dx - dt*ufl.dot(b_V*yu,ufl.grad(phiu))*ufl.dx
bilinear_formu = fem.form(a_hu)

# Variational Problem (RHS).
Lu = (y_0u + dt*fem.Constant(domain, 0.0))*phiu*ufl.dx
linear_formu = fem.form(Lu)

# Assemble FEM matrices.
Cu = femPetsc.assemble_matrix(bilinear_formu)
Cu.assemble()
du = femPetsc.create_vector(linear_formu)

# Create solver.
solveru = PETSc.KSP().create(domain.comm)
solveru.setOperators(Cu)
solveru.setType(PETSc.KSP.Type.PREONLY)
solveru.getPC().setType(PETSc.PC.Type.LU)

# Whole time intervall.
for i in range(num_steps):
  t += dt

  # Update the right hand side reusing the initial vector
  with du.localForm() as loc_b:
    loc_b.set(0)
  femPetsc.assemble_vector(du, linear_formu)

  # Apply Dirichlet boundary condition to the vector
  femPetsc.apply_lifting(du, [bilinear_formu], [[bc]])
  du.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
  femPetsc.set_bc(du, [bc])

  # Solve linear problem
  solveru.solve(du, y_nu.vector)
  y_nu.x.scatter_forward()

  # Update solution at previous time step
  y_0u.x.array[:] = y_nu.x.array

  # Write solution to file
  xdmfu.write_function(y_nu, t)
xdmfu.close()

In [ ]:
#Install control package to calulate the solution of the riccati equations

#X, L, K = control.care(Ad, B, Q_wb, R, np.zeros((n_wb, m)), M_wb)
#K = np.asarray(K)